In [1]:
import urllib
import geopandas as gpd

In [2]:
# input similar to aodn catalog
geom= 'POLYGON ((115.0000000000000000 -33.0000000000000000, 115.0000000000000000 -31.5000000000000000, 115.8000030517578125 -31.5000000000000000, 115.8000030517578125 -33.0000000000000000, 115.0000000000000000 -33.0000000000000000))'
time_min,time_max = '2021-02-03','2021-02-10'

In [3]:
# inspect altimetry data
params={}
params['REQUEST'] = 'GetFeature'
params['SERVICE'] = 'WFS'
params['VERSION'] = '1.0.0'
params['typeName'] = 'srs_surface_waves_altimetry_map'
params['CQL_FILTER'] = f"INTERSECTS(geom, {geom})"
wfs_url = 'http://geoserver-portal.aodn.org.au/geoserver/ows'
url = f'{wfs_url}?{urllib.parse.urlencode(params)}'
df = gpd.read_file(url)
df.columns # n

/home/rfonsecadasilva/anaconda3/envs/rompy-dev/lib/python3.7/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Index(['gml_id', 'timeseries_id', 'platform_code', 'time_coverage_start',
       'time_coverage_end', 'geospatial_lat_min', 'geospatial_lat_max',
       'geospatial_lon_min', 'geospatial_lon_max', 'lon_cell', 'lat_cell',
       'file_url', 'size', 'geometry'],
      dtype='object')

No 'TIME' field present --> time filter won't work; but time_coverage_start and time_coverage_end can be used as filters (see next cells)

In [4]:
# inspect sar data
params['typeName'] = 'srs_surface_waves_sar_fv00_trajectory_map'
params['CQL_FILTER'] = f"INTERSECTS(geom, {geom})"
url = f'{wfs_url}?{urllib.parse.urlencode(params)}'
df = gpd.read_file(url)
df.columns # no time

Index(['gml_id', 'trajectory_id', 'url', 'size', 'platform_code',
       'wavnum_type', 'time_coverage_start', 'time_coverage_end', 'geometry'],
      dtype='object')

No 'TIME' field present for sar

In [5]:
# now applying time filter - note different dates, otherwise no data available
time_min,time_max = '2020-02-03','2021-02-10'
params['typeName'] = 'srs_surface_waves_altimetry_map'
params['CQL_FILTER'] = f"INTERSECTS(geom, {geom}) AND time_coverage_start <= '{time_max}' AND time_coverage_end >= '{time_min}'"
url = f'{wfs_url}?{urllib.parse.urlencode(params)}'
df = gpd.read_file(url)
df.head

<bound method NDFrame.head of                                               gml_id  timeseries_id  \
0  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2810733   
1  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2886209   
2  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2784024   
3  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2752089   
4  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2833622   
5  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2895810   
6  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2852838   
7  srs_surface_waves_altimetry_map.fid-5f1bee80_1...        2800227   

  platform_code  time_coverage_start    time_coverage_end  geospatial_lat_min  \
0         SARAL  2013-03-17T22:47:05  2020-12-27T09:52:18          -32.999260   
1   SENTINEL-3A  2016-03-09T14:41:41  2020-12-29T01:56:55          -32.999882   
2       JASON-3  2016-02-17T08:53:25  2020-12-27T06:29:51          -31.990362   
3     